# Imports

In [2]:
%pip install --upgrade tiktoken
%pip install --upgrade sentencepiece
%pip install --upgrade tensorflow
%pip install --upgrade tf_keras
%pip install --upgrade transformers
%pip install ml_dtypes==0.5.1
%pip install --upgrade minicons

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import tensorflow
import pandas as pd
import numpy as np
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel, TFBertForMaskedLM, BertTokenizer
import torch
import huggingface_hub as hgh
from minicons import scorer
import operator

# Login (for HuggingFace)

In [4]:
login_token_file = open("access_items/hg_token.txt", "r")
token = login_token_file.readline()
login_token_file.close()
hgh.login(token)

[BanglaBERT: Bengali Mask Language Model for Bengali Language Understanding](https://github.com/sagorbrur/bangla-bert) (Sagor Sarker, 2020)

In [22]:
# load model + tokenizer
tokenizer = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
model = TFBertForMaskedLM.from_pretrained("sagorsarker/bangla-bert-base")

In [ ]:
# load data
# location of corpus data is as given, but not included in github due to possible copyright issues
sentences = list(pd.read_csv('assamese_corpus\\asm_wikipedia_2021_30K\\asm_wikipedia_2021_30K\\asm_wikipedia_2021_30K-sentences.txt', quotechar='"', sep='\t',header=None)[1])

In [24]:
# tokenize data
inputs = tokenizer(list(sentences), max_length=100, truncation=True, padding='max_length', return_tensors='tf')

KeyboardInterrupt: 

https://www.analyticsvidhya.com/blog/2022/09/fine-tuning-bert-with-masked-language-modeling/

In [18]:
tensorflow.experimental.numpy.experimental_enable_numpy_behavior()

In [19]:
inputs['labels'] = inputs['input_ids']

In [20]:

inp_ids = []
lbs = []
idx = 0
for inp in inputs.input_ids.numpy():
    actual_tokens = list(set(range(100)) - 
                         set(np.where((inp == 101) | (inp == 102) 
                            | (inp == 0))[0].tolist()))
    #We need to select 15% random tokens from the given list
    num_of_token_to_mask = int(len(actual_tokens)*0.15)
    token_to_mask = np.random.choice(np.array(actual_tokens), 
                                     size=num_of_token_to_mask, 
                                     replace=False).tolist()
    #Now we have the indices where we need to mask the tokens
    inp[token_to_mask] = 103
    inp_ids.append(inp)
    idx += 1
inp_ids = tensorflow.convert_to_tensor(inp_ids)
inputs['input_ids'] = inp_ids

In [25]:
with tensorflow.device('/GPU:0'):
    model.compile(optimizer="adam",loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
    history = model.fit([inputs.input_ids,inputs.attention_mask],inputs.labels,verbose=1,batch_size=8,epochs=10)

Epoch 1/10
  31/3556 [..............................] - ETA: 6:13:24 - loss: 5.4543

KeyboardInterrupt: 